## Imports
---

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, RandomForestRegressor, AdaBoostRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split, cross_val_predict

#### Load the season long data set for RB statistics and DK points from weeks 1 through 9
---

In [2]:
# read in data set
rb_game = pd.read_csv('../data/RB_season_9.csv', index_col=[0])
rb_game

,player,pos,team,opp_home,opp,game,week,day,rush_att,rush_yds,...,dk_pt,pass_cmp,pass_att,pass_yds,pass_td,pass_int,fmb,team_win,team_score,opp_score
0,Derrick Henry,RB,TEN,1.0,SEA,2,2,Sun,35,182,...,50.7,0,0,0,0,0,0,1,33,30
1,Aaron Jones,RB,GNB,0.0,DET,2,2,Mon,17,67,...,41.5,0,0,0,0,0,0,1,35,17
2,James Conner,RB,ARI,1.0,SFO,9,9,Sun,21,96,...,40.3,0,0,0,0,0,1,1,31,17
3,Derrick Henry,RB,TEN,0.0,BUF,6,6,Mon,20,143,...,38.6,0,0,0,0,0,0,1,34,31
4,Jonathan Taylor,RB,IND,0.0,NYJ,9,9,Thu,19,172,...,37.0,0,0,0,0,0,1,1,45,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
656,Peyton Barber,RB,LVR,1.0,LAC,4,4,Mon,1,0,...,0.0,0,0,0,0,0,0,0,14,28
657,Corey Clement,RB,DAL,0.0,PHI,3,3,Mon,3,0,...,0.0,0,0,0,0,0,0,1,41,21
658,Justin Jackson,RB,LAC,1.0,KAN,3,3,Sun,2,0,...,0.0,0,0,0,0,0,0,1,30,24
659,Benny Snell Jr.,RB,PIT,1.0,GNB,4,4,Sun,1,0,...,0.0,0,0,0,0,0,0,0,17,27


In [3]:
# take note of the columns before dropping most of them...
rb_game.columns

Index(['player', 'pos', 'team', 'opp_home', 'opp', 'game', 'week', 'day',
       'rush_att', 'rush_yds', 'rush_yda', 'rush_td', 'rec_tgt', 'rec',
       'rec_yds', 'rec_ydr', 'rec_td', 'rec_pct', 'rec_ydtgt', 'dk_pt',
       'pass_cmp', 'pass_att', 'pass_yds', 'pass_td', 'pass_int', 'fmb',
       'team_win', 'team_score', 'opp_score'],
      dtype='object')

In [4]:
# we will only be using team, opponent, location, and which week the game was played
X = rb_game[['team','opp_home','opp','week']]
y = rb_game['dk_pt']

In [5]:
# dummify the offensive team
X = pd.get_dummies(X, columns=['team'], drop_first=True)
X

,opp_home,opp,week,team_ATL,team_BAL,team_BUF,team_CAR,team_CHI,team_CIN,team_CLE,...,team_NWE,team_NYG,team_NYJ,team_PHI,team_PIT,team_SEA,team_SFO,team_TAM,team_TEN,team_WAS
0,1.0,SEA,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0.0,DET,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.0,SFO,9,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.0,BUF,6,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0.0,NYJ,9,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
656,1.0,LAC,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
657,0.0,PHI,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
658,1.0,KAN,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
659,1.0,GNB,4,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [6]:
# dummify the opponent
X = pd.get_dummies(X, columns=['opp'], drop_first=True)
X

,opp_home,week,team_ATL,team_BAL,team_BUF,team_CAR,team_CHI,team_CIN,team_CLE,team_DAL,...,opp_NWE,opp_NYG,opp_NYJ,opp_PHI,opp_PIT,opp_SEA,opp_SFO,opp_TAM,opp_TEN,opp_WAS
0,1.0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0.0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.0,9,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0.0,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.0,9,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
656,1.0,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
657,0.0,3,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
658,1.0,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
659,1.0,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# take note of the shape before splitting the data
X.shape, y.shape

((661, 64), (661,))

In [8]:
# split our data into training and testing with a 70/30 split and random state of 42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((462, 64), (462,), (199, 64), (199,))

#### Look at linear regression and regularization models first
---

In [9]:
# set up and run Linear Regression model
lr = LinearRegression()
lr.fit(X_train, y_train)
lr.score(X_train, y_train), lr.score(X_test, y_test)

(0.15705033222719933, -0.1313491127206916)

In [10]:
# let's run ridge with baseline parameters
ridge = Ridge()
ridge.fit(X_train, y_train)
ridge.score(X_train, y_train), ridge.score(X_test, y_test)

(0.1547588250995111, -0.10984390479690842)

In [11]:
# now cross validate a Ridge model while searching for the best alpha and score over R2
r_alphas = np.logspace(0, 5, 100)
ridge_cv = RidgeCV(alphas=r_alphas, scoring='r2', cv=5)
ridge_cv.fit(X_train, y_train)
ridge_cv.score(X_train, y_train), ridge_cv.score(X_test, y_test)

(0.05902986233940677, -0.015982261321979108)

In [12]:
# let's run LASSO with baseline parameters
lasso = Lasso()
lasso.fit(X_train, y_train)
lasso.score(X_train, y_train), lasso.score(X_test, y_test)

(0.0011099357079323857, -0.021178969036831807)

In [13]:
# now cross validate a LASSO model while searching for the best alpha
l_alphas = np.logspace(-3, 3, 100)
lasso_cv = LassoCV(alphas=l_alphas, cv=5)
lasso_cv.fit(X_train, y_train)
lasso_cv.score(X_train, y_train), lasso_cv.score(X_test, y_test)

(0.05763232421038389, -0.029700153697818976)

---
#### The linear and regularization models are largely overfit and perform terribly on testing data. The cross validated Ridge model looks the best based on its R2 scores with the lowest testing score.
---

#### Take a look at KNN regression
---

In [14]:
# run baseline KNN regression model
knr = KNeighborsRegressor()
knr.fit(X_train, y_train)
knr.score(X_train, y_train), lr.score(X_test, y_test)

(0.21005537869490998, -0.1313491127206916)

In [15]:
# setup a range of k neighbors and distance metric to test against
knr_params = {
    'n_neighbors': range(1, 51, 1),
    'metric': ['euclidean', 'manhattan']
}

# grid search on these metrics
knr_gridsearch = GridSearchCV(estimator=KNeighborsRegressor(),
                              param_grid=knr_params,
                              cv=5)

# fit the model based on the parameters above
knr_gridsearch.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsRegressor(),
             param_grid={'metric': ['euclidean', 'manhattan'],
                         'n_neighbors': range(1, 51)})

In [16]:
# display the best parameters
knr_gridsearch.best_params_

{'metric': 'manhattan', 'n_neighbors': 49}

In [17]:
# let's look at the R2 scores for this model
knr_gridsearch.score(X_train, y_train), knr_gridsearch.score(X_test, y_test)

(0.032727851250486495, -0.05013831717417516)

---
#### The baseline model is overfit based on the negative R2 testing score. After searching for and using the best parameters, we still see an overfit model with negative R2 score for testing data.
---

#### How does a Decision Tree look? Consider Random Forest and AdaBoost
---

In [18]:
# let's look at the baseline Random Forest before grid searching
rfr = RandomForestRegressor()
rfr.fit(X_train, y_train)
rfr.score(X_train, y_train), rfr.score(X_test, y_test)

(0.49036670142221184, -0.5080223313674037)

In [19]:
# set up a range of parameter to check against
grid = {
    'n_estimators': [100, 150, 200, 250, 300],
    'max_depth': [None, 3, 5, 7],
    'min_samples_split': [2, 3, 5, 7]
}

# run the grid search
gs = GridSearchCV(rfr, param_grid=grid, cv=5)

# fit the model based on these parameters
gs.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [None, 3, 5, 7],
                         'min_samples_split': [2, 3, 5, 7],
                         'n_estimators': [100, 150, 200, 250, 300]})

In [20]:
# what are the best parameters?
gs.best_params_

{'max_depth': 3, 'min_samples_split': 7, 'n_estimators': 150}

In [21]:
# what are the R2 scores for the data?
gs.score(X_train, y_train), gs.score(X_test, y_test)

(0.12111020576658893, -0.04580962376955844)

In [22]:
# set up and run the AdaBoost model using default values
abr = AdaBoostRegressor(random_state=42, n_estimators=100)
abr.fit(X_train, y_train)
abr.score(X_train, y_train), abr.score(X_test, y_test)

(0.028435509919172763, -0.04397921441593522)

---
#### The baseline model for Random Forest has the best training score of all models checked. However, the testing score is massively negative and indicates overfitting. When grid searching and cross validating, we see the training and testing score come much closer together, but we still see a negative testing R2 score. AdaBoost did not perform better than Random Forest, so we will not pursue this any further considering its inherent time consumption.
---

#### Load in Week 10 player list to run predictions against
---

In [23]:
# read in the week 10 data
rb_pred = pd.read_csv('../data/week_10_RB_preds.csv')
rb_pred.head()

,player,team,opp_home,opp,week
0,Najee Harris,PIT,0,DET,10
1,Jonathan Taylor,IND,0,JAX,10
2,Austin Ekeler,LAC,0,MIN,10
3,Ezekiel Elliott,DAL,0,ATL,10
4,Alvin Kamara,NOR,1,TEN,10


In [24]:
# drop the player name and use this dataframe to run predictions on
rb_pred_test = rb_pred.drop(columns='player')
rb_pred_test.head()

,team,opp_home,opp,week
0,PIT,0,DET,10
1,IND,0,JAX,10
2,LAC,0,MIN,10
3,DAL,0,ATL,10
4,NOR,1,TEN,10


In [25]:
# take note of the shape
rb_pred_test.shape

(102, 4)

In [26]:
# dummify the offensive team
rb_pred_test = pd.get_dummies(rb_pred_test, columns=['team'], drop_first=True)
rb_pred_test.head()

,opp_home,opp,week,team_ATL,team_BAL,team_BUF,team_CAR,team_CLE,team_DAL,team_DEN,...,team_NOR,team_NWE,team_NYJ,team_PHI,team_PIT,team_SEA,team_SFO,team_TAM,team_TEN,team_WAS
0,0,DET,10,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,JAX,10,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,MIN,10,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,ATL,10,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,TEN,10,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [27]:
# dummify the opponent
rb_pred_test = pd.get_dummies(rb_pred_test, columns=['opp'], drop_first=True)
rb_pred_test.head()

,opp_home,week,team_ATL,team_BAL,team_BUF,team_CAR,team_CLE,team_DAL,team_DEN,team_DET,...,opp_NOR,opp_NWE,opp_NYJ,opp_PHI,opp_PIT,opp_SEA,opp_SFO,opp_TAM,opp_TEN,opp_WAS
0,0,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,10,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [28]:
# now observe the shape and realize we need to get to 64 columns
rb_pred_test.shape

(102, 56)

In [29]:
# which columns are we missing
rb_pred_test.columns

Index(['opp_home', 'week', 'team_ATL', 'team_BAL', 'team_BUF', 'team_CAR',
       'team_CLE', 'team_DAL', 'team_DEN', 'team_DET', 'team_GNB', 'team_IND',
       'team_JAX', 'team_KAN', 'team_LAC', 'team_LAR', 'team_LVR', 'team_MIA',
       'team_MIN', 'team_NOR', 'team_NWE', 'team_NYJ', 'team_PHI', 'team_PIT',
       'team_SEA', 'team_SFO', 'team_TAM', 'team_TEN', 'team_WAS', 'opp_ATL',
       'opp_BAL', 'opp_BUF', 'opp_CAR', 'opp_CLE', 'opp_DAL', 'opp_DEN',
       'opp_DET', 'opp_GNB', 'opp_IND', 'opp_JAX', 'opp_KAN', 'opp_LAC',
       'opp_LAR', 'opp_LVR', 'opp_MIA', 'opp_MIN', 'opp_NOR', 'opp_NWE',
       'opp_NYJ', 'opp_PHI', 'opp_PIT', 'opp_SEA', 'opp_SFO', 'opp_TAM',
       'opp_TEN', 'opp_WAS'],
      dtype='object')

In [30]:
# of course, we are missing the teams that are on bye week
rb_pred_test['team_CHI'] = 0
rb_pred_test['opp_CHI'] = 0
rb_pred_test['team_CIN'] = 0
rb_pred_test['opp_CIN'] = 0
rb_pred_test['team_HOU'] = 0
rb_pred_test['opp_HOU'] = 0
rb_pred_test['team_NYG'] = 0
rb_pred_test['opp_NYG'] = 0

#### Apply predictions to the Player Matchups for Week 10
---

In [31]:
# add columns for all of the predictions of interest (best performing models) if we can even say that...
rb_pred['pass_preds_ridgeCV'] = ridge_cv.predict(rb_pred_test)
rb_pred['pass_preds_knr_gs'] = knr_gridsearch.predict(rb_pred_test)
rb_pred['pass_preds_rfr'] = rfr.predict(rb_pred_test)
rb_pred['pass_preds_rfr_gs'] = gs.predict(rb_pred_test)

In [32]:
rb_pred.sort_values(by='pass_preds_rfr', ascending=False).head(10)

,player,team,opp_home,opp,week,pass_preds_ridgeCV,pass_preds_knr_gs,pass_preds_rfr,pass_preds_rfr_gs
99,Devine Ozigbo,JAX,1,IND,10,11.546204,10.585714,16.536876,11.698962
87,Dare Ogunbowale,JAX,1,IND,10,11.546204,10.585714,16.536876,11.698962
12,James Robinson,JAX,1,IND,10,11.546204,10.585714,16.536876,11.698962
46,Carlos Hyde,JAX,1,IND,10,11.546204,10.585714,16.536876,11.698962
96,Dwayne Washington,NOR,1,TEN,10,11.036828,10.022449,14.612350,9.382721
4,Alvin Kamara,NOR,1,TEN,10,11.036828,10.022449,14.612350,9.382721
81,Alex Armah,NOR,1,TEN,10,11.036828,10.022449,14.612350,9.382721
48,Mark Ingram,NOR,1,TEN,10,11.036828,10.022449,14.612350,9.382721
27,Alex Collins,SEA,1,GNB,10,10.351774,8.516327,14.200250,9.394248
72,Rashaad Penny,SEA,1,GNB,10,10.351774,8.516327,14.200250,9.394248


---
#### Remember that the top 5 worst defenses by DK points allowed are NYJ (by a lot) > DET > SFO > CIN > PHI > LAR > NYG > ATL > BAL 
note: SFO and LAR played Monday night of which we do not have data and CIN and NYG are on bye in week 10. 

#### RidgeCV has 2 of those top 5 in its top 10 for rushing points (did predict BUF as 1st although they rank the best against rushing and IND as 2nd although they rank 3rd best against rushing)
#### KNNR GS has 2 of those top 5 in its top 10 for rushing points 
#### RndFor has 2 of those top 5 in its top 10 for rushing points (did predict IND as 2nd although they rank 3rd best against rushing)
#### RF GS has 2 of those top 5 in its top 10 for rushing points (did predict BUF as 1st although they rank the best against rushing and IND as 2nd although they rank 3rd best against rushing)

#### If we are using the null model as a baseline like above, then we should consider KNNR GS as our best model to compare with the actual results from week 10 taking into account that it did not predict 2 of the best teams against rushing in its top 5.
---

#### Load in Week 10 player results and null model to compare predictions against
---

In [33]:
# read in week 10 RB results and simplify columns
rb_results = pd.read_csv('../data/week_10_RB_results.csv', index_col=[0])
rb_results = rb_results[['player','team','opp_home','opp','DK_pt']]
rb_results.head()

,player,team,opp_home,opp,DK_pt
0,Darrel Williams,KAN,1.0,LVR,32.4
1,Rhamondre Stevenson,NWE,0.0,CLE,30.4
2,Jonathan Taylor,IND,0.0,JAX,27.6
3,AJ Dillon,GNB,0.0,SEA,26.8
4,Christian McCaffrey,CAR,1.0,ARI,26.1


In [34]:
# read in null model (season average) versus RB data set and simplify columns
rb_null = pd.read_csv('../data/DEF_RB.csv', index_col=[0])
rb_null = rb_null[['opp','DK_ptg']]
rb_null.head()

,opp,DK_ptg
0,NYJ,40.6
1,DET,31.2
7,SFO,31.0
6,CIN,29.3
3,PHI,29.3


In [35]:
# merge the data frames on opponent and only view the following columns
rb_final = rb_results.merge(rb_null, on='opp', how='outer')
rb_final = pd.DataFrame(data=rb_final, columns=['player','team','DK_pt','opp_home','opp','DK_ptg'])
rb_final.head()

,player,team,DK_pt,opp_home,opp,DK_ptg
0,Darrel Williams,KAN,32.4,1.0,LVR,24.5
1,Jerick McKinnon,KAN,4.2,1.0,LVR,24.5
2,Derrick Gore,KAN,1.9,1.0,LVR,24.5
3,Rhamondre Stevenson,NWE,30.4,0.0,CLE,21.0
4,Brandon Bolden,NWE,10.0,0.0,CLE,21.0


In [36]:
# now merge with the predictions on the common columns between them
rb_final = rb_final.merge(rb_pred, on=['player','team','opp_home','opp'], how='outer')
rb_final.head()

,player,team,DK_pt,opp_home,opp,DK_ptg,week,pass_preds_ridgeCV,pass_preds_knr_gs,pass_preds_rfr,pass_preds_rfr_gs
0,Darrel Williams,KAN,32.4,1.0,LVR,24.5,10.0,9.572618,8.536735,3.944736,8.637149
1,Jerick McKinnon,KAN,4.2,1.0,LVR,24.5,10.0,9.572618,8.536735,3.944736,8.637149
2,Derrick Gore,KAN,1.9,1.0,LVR,24.5,10.0,9.572618,8.536735,3.944736,8.637149
3,Rhamondre Stevenson,NWE,30.4,0.0,CLE,21.0,10.0,9.473981,9.469388,5.426617,9.212250
4,Brandon Bolden,NWE,10.0,0.0,CLE,21.0,10.0,9.473981,9.469388,5.426617,9.212250


In [40]:
# view actual week 10 results with the model predictions
rb_final.sort_values(by='DK_pt', ascending=False).head(50)

,player,team,DK_pt,opp_home,opp,DK_ptg,week,pass_preds_ridgeCV,pass_preds_knr_gs,pass_preds_rfr,pass_preds_rfr_gs
0,Darrel Williams,KAN,32.4,1.0,LVR,24.5,10.0,9.572618,8.536735,3.944736,8.637149
3,Rhamondre Stevenson,NWE,30.4,0.0,CLE,21.0,10.0,9.473981,9.469388,5.426617,9.212250
6,Jonathan Taylor,IND,27.6,0.0,JAX,23.3,10.0,10.908999,10.738776,11.870300,11.169338
8,AJ Dillon,GNB,26.8,0.0,SEA,25.8,10.0,10.588243,9.787755,11.786174,9.251574
11,Christian McCaffrey,CAR,26.1,1.0,ARI,21.1,10.0,10.635576,9.032653,9.758467,9.230452
15,D'Ernest Johnson,CLE,22.7,1.0,NWE,24.7,10.0,10.074853,8.710204,7.245217,9.196167
20,Antonio Gibson,WAS,21.8,0.0,TAM,22.5,10.0,8.726127,9.253061,4.755994,8.772035
17,Ezekiel Elliott,DAL,21.8,0.0,ATL,27.6,10.0,10.695009,11.073469,10.209083,9.452327
27,Mark Ingram,NOR,20.8,1.0,TEN,21.3,10.0,11.036828,10.022449,14.612350,9.382721
23,Dalvin Cook,MIN,20.8,1.0,LAC,25.6,10.0,9.521207,8.536735,9.432517,9.145380


---
#### Looking at the top 10 RB points for week 10, we see that only 1 of the top 5 null model defensive points allowed made the top 10 actual scores. The top actual score was against a defense that allowed the 17th most points to QBs. Let's list out our top 5 by our top models and see where it ranked with actual QB point totals.

|KNNR GS||RndFor||
|---|---|---|---|
|Pred Opp|Actual Opp|Pred Opp|Actual Opp|
|MIN - 11.5|19th - 14.9|IND - 16.7|14th - 18.4| 
|CAR - 11.1|18th - 15.4|TEN - 15.0|9th - 20.8|
|ATL - 11.1|8th - 21.8|JAX - 14.1|3rd - 27.6|
|DET - 11.0|11th - 20.3|GNB - 13.1|39th - 5.9|
|JAX - 10.7|3rd - 27.6|DET - 11.9|11th - 20.3|

#### To be fair BUF utilized 3 RBs that totaled 39.6 points which was just 1 point off from NYJ league best 40.6 points allowed to rushing on average. Both models picked a 3rd best actual finish in their top 5, while each only predicting one more top 10 finisher in the predictions' top 5. As mentioned above, I wanted to exclude the models that picked the best rushing defenses in their top 10. The reality is the 15th best actual RB did play against BUF (1st against rush) and the 14th best actual RB did play against IND (3rd against rush).
---